In [1]:
# Read the phenotype info

import pandas as pd

csvname = '/run/media/anton/Seagate Portable Drive/UKB_MRI_data_11092022/UKB_Phenotypes/project3_Anton/DataSet_11162022.csv'
df = pd.read_csv(csvname)
df

,f.eid,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,f.21022.0.0,f.22001.0.0,f.25750.2.0,f.25750.3.0,f.25751.2.0,f.25751.3.0,f.25752.2.0,f.25752.3.0,f.25753.2.0,f.25753.3.0,f.25754.2.0,f.25754.3.0,f.25755.2.0,f.25755.3.0
0,1000010,50.0,NaN,NaN,NaN,50.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000028,52.0,NaN,NaN,NaN,52.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000034,56.0,NaN,NaN,NaN,56.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000045,43.0,NaN,NaN,NaN,43.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000052,61.0,NaN,NaN,NaN,61.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502481,6024820,68.0,NaN,NaN,NaN,68.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502482,6024833,61.0,NaN,NaN,NaN,61.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502483,6024847,64.0,NaN,NaN,NaN,64.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502484,6024859,60.0,NaN,NaN,NaN,60.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# Get good filtered timeseries

import pickle
import numpy as np

funcdir = '/home/anton/Documents/Tulane/Research/UKBB/Cypress/func/'
ts = pickle.load(open(f'{funcdir}/../ts/ukb_ts_20227_0-600_good_filt.pkl', 'rb'))

print(len(ts))

556


In [10]:
df[df['f.eid'] == 1000076]['f.22001.0.0']

6    1.0
Name: f.22001.0.0, dtype: float64

In [4]:
from math import isnan

ps = []
age = []
sex = []

a,b = np.triu_indices(264,1)

for sub,scans in ts.items():
    for val in scans:
        ses, task, ts0 = val
        ag = float(df[df['f.eid'] == int(sub)]['f.21003.0.0'])
        s = float(df[df['f.eid'] == int(sub)]['f.22001.0.0'])
        if isnan(ag) or isnan(s):
            print(sub,ses)
            continue
        p = np.corrcoef(ts0)[a,b]
        ps.append(p)
        age.append(ag)
        sex.append(int(s))
        
ps = np.stack(ps)
age = np.array(age)
sex = np.array(sex)
        
print([item.shape for item in [ps, age, sex]])

1006604 2
1009819 2
1011256 2
1013566 2
1015850 2
1028465 2
1037381 2
1041164 2
1053570 2
1054850 2
1059329 2
1059905 2
1059905 3
1060674 2
[(586, 34716), (586,), (586,)]


In [37]:
# Angle basis

# Angle estimate

import torch
import torch.nn as nn

import numpy as np
import matplotlib.pyplot as plt

pi = 3.14
pi2 = 2*pi

def tonp(t):
    return t.detach().cpu().numpy()

class AngleBasis(nn.Module):
    def __init__(self, mixn):
        super(AngleBasis, self).__init__()
        self.mixn = mixn
        self.thetas = nn.Parameter((pi*torch.rand(self.mixn,264)+pi/2).float().cuda())
        self.jitter = nn.Parameter(torch.ones(self.mixn,264).float().cuda())
        
    def project(self):
        with torch.no_grad():
            self.jitter[self.jitter < 0] = 0
            self.jitter[self.jitter > 1] = 1
        
    def phases(self):
        t0 = self.thetas.unsqueeze(2)
        t1 = self.thetas.unsqueeze(1)
        return t0-t1
    
    def jit(self):
        j0 = self.jitter.unsqueeze(2)
        j1 = self.jitter.unsqueeze(1)
        return j0*j1
    
    def ps(self, jitter=True):
        t = self.phases()
        p = torch.cos(t)
        if jitter:
            j = self.jit()
            p = j*p
        return p
    
    def dump(self):
        return dict(mixn=self.mixn, thetas=tonp(self.thetas), jitter=tonp(self.jitter))
    
    def psum(self):
        return torch.mean(self.ps(), axis=0)
    
    def pvec(self):
        a,b = np.triu_indices(264,1)
        p = self.psum()
        return p[a,b]
    
print('Complete')

Complete


In [39]:
nepochs = 3000
pperiod = 500

def fromnp(x):
    return torch.from_numpy(x).float().cuda()

bases = dict()
i = 0

a,b = np.triu_indices(264,1)

for sub,scans in ts.items():
    for val in scans:
        ses, task, ts0 = val
        
        p = np.corrcoef(ts0)[a,b]
        x = fromnp(p)

        min_loss = 10
        sav = None

        for _ in range(2):
            basis = AngleBasis(5)
            optim = torch.optim.Adam(basis.parameters(), lr=1e-2, weight_decay=0)

            for e in range(nepochs):
                optim.zero_grad()
                xhat = basis.pvec()
                loss = rmse(xhat, x)
                loss.backward()
                optim.step()
                basis.project()
                if e == nepochs-1 or e % pperiod == 1:
                    print(f'{e} {float(loss)}')

            if loss < min_loss:
                print('Saving')
                min_loss = float(loss)
                sav = basis
                
        sub_task_ses = f'{sub}_{task}_{ses}'
        
        bases[sub_task_ses] = sav.dump()
        i += 1

        print(f'Finished {i} {sub_task_ses}')
        
print('Complete')

1 0.34458571672439575
501 0.04675526171922684
1001 0.04525725543498993
1501 0.04501335322856903
2001 0.044966522604227066
2501 0.044958438724279404
2999 0.044955018907785416
Saving
1 0.3488154709339142
501 0.04696163162589073
1001 0.043589260429143906
1501 0.04315844178199768
2001 0.04299333691596985
2501 0.0429537370800972
2999 0.04292751103639603
Saving
Finished 1 1000076_20227_2
1 0.3381653130054474
501 0.05248251184821129
1001 0.05046159401535988
1501 0.047827187925577164
2001 0.04546977952122688
2501 0.04412754252552986
2999 0.043833933770656586
Saving
1 0.33264032006263733
501 0.05054544657468796
1001 0.04834716394543648
1501 0.04831025004386902
2001 0.04829428344964981
2501 0.04809896647930145
2999 0.047496549785137177
Finished 2 1000076_20227_3
1 0.385397344827652
501 0.0738639384508133
1001 0.07153446972370148
1501 0.06773949414491653
2001 0.06416560709476471
2501 0.05926552414894104
2999 0.05792314186692238
Saving
1 0.40025949478149414
501 0.07631386816501617
1001 0.069645904

1001 0.053942129015922546
1501 0.05275925248861313
2001 0.052088480442762375
2501 0.051535364240407944
2999 0.05083458498120308
Saving
Finished 22 1001382_20227_2
1 0.3325229585170746
501 0.03139295428991318
1001 0.029666341841220856
1501 0.028676141053438187
2001 0.02842753939330578
2501 0.028198564425110817
2999 0.02780989371240139
Saving
1 0.3330112397670746
501 0.03486257046461105
1001 0.03128907084465027
1501 0.029974142089486122
2001 0.028103763237595558
2501 0.027317170053720474
2999 0.02704918198287487
Saving
Finished 23 1001464_20227_2
1 0.35064074397087097
501 0.033210840076208115
1001 0.030417971312999725
1501 0.028226085007190704
2001 0.0273770522326231
2501 0.026997972279787064
2999 0.02699180133640766
Saving
1 0.344613641500473
501 0.03263372555375099
1001 0.029515521600842476
1501 0.029091544449329376
2001 0.028536129742860794
2501 0.028129570186138153
2999 0.028105424717068672
Finished 24 1001464_20227_3
1 0.328111469745636
501 0.05662722513079643
1001 0.051619451493024

1001 0.045226968824863434
1501 0.043478891253471375
2001 0.04296749830245972
2501 0.041969940066337585
2999 0.041947610676288605
Saving
1 0.3278388977050781
501 0.050225745886564255
1001 0.04779946058988571
1501 0.04431698098778725
2001 0.043281883001327515
2501 0.0419081412255764
2999 0.041256990283727646
Saving
Finished 44 1003312_20227_2
1 0.3265620172023773
501 0.03705863282084465
1001 0.03052285686135292
1501 0.029462415724992752
2001 0.02937578596174717
2501 0.029361560940742493
2999 0.029200183227658272
Saving
1 0.3285958170890808
501 0.03478608652949333
1001 0.03230389207601547
1501 0.031001528725028038
2001 0.029578780755400658
2501 0.02901771105825901
2999 0.028893981128931046
Saving
Finished 45 1003547_20227_2
1 0.35634511709213257
501 0.06432908028364182
1001 0.06083090603351593
1501 0.05987716093659401
2001 0.058010533452034
2501 0.056206025183200836
2999 0.05480846017599106
Saving
1 0.361613005399704
501 0.06237375736236572
1001 0.059878166764974594
1501 0.058851350098848

1501 0.04051150381565094
2001 0.03982381522655487
2501 0.03892313316464424
2999 0.03758486732840538
Saving
Finished 65 1005426_20227_2
1 0.34509947896003723
501 0.052906498312950134
1001 0.04915910214185715
1501 0.04816775023937225
2001 0.0465795062482357
2501 0.04519115388393402
2999 0.04466633498668671
Saving
1 0.33617332577705383
501 0.05114925652742386
1001 0.049656014889478683
1501 0.049518484622240067
2001 0.04871735721826553
2501 0.047771528363227844
2999 0.046246644109487534
Finished 66 1005463_20227_2
1 0.38448482751846313
501 0.07398393005132675
1001 0.06732313334941864
1501 0.06476263701915741
2001 0.06243911758065224
2501 0.060211535543203354
2999 0.05875348672270775
Saving
1 0.37654632329940796
501 0.06505774706602097
1001 0.06263387203216553
1501 0.061943911015987396
2001 0.061616476625204086
2501 0.05972808599472046
2999 0.05740029737353325
Saving
Finished 67 1005637_20227_2
1 0.3428392708301544
501 0.05531581863760948
1001 0.05198127031326294
1501 0.05102815106511116
20

1001 0.041170019656419754
1501 0.03978358581662178
2001 0.03908374905586243
2501 0.03844692185521126
2999 0.03842870891094208
Saving
1 0.33559417724609375
501 0.04725244268774986
1001 0.04334764555096626
1501 0.041610829532146454
2001 0.041588667780160904
2501 0.041563354432582855
2999 0.04156326502561569
Finished 87 1007748_20227_2
1 0.32681238651275635
501 0.045944392681121826
1001 0.042228005826473236
1501 0.041243936866521835
2001 0.04001189395785332
2501 0.039856791496276855
2999 0.039847224950790405
Saving
1 0.32719340920448303
501 0.04526321589946747
1001 0.0418829508125782
1501 0.04027700796723366
2001 0.039191197603940964
2501 0.038977377116680145
2999 0.03866534307599068
Saving
Finished 88 1007883_20227_2
1 0.3292391896247864
501 0.054586011916399
1001 0.04942658543586731
1501 0.048855021595954895
2001 0.048839423805475235
2501 0.04883751645684242
2999 0.04883705452084541
Saving
1 0.3280881345272064
501 0.052294861525297165
1001 0.0500275120139122
1501 0.04907674342393875
200

...

501 0.04806354269385338
1001 0.04584610089659691
1501 0.044474028050899506
2001 0.04177476093173027
2501 0.04121026024222374
2999 0.04054739698767662
Saving
Finished 495 1054850_20227_2
1 0.3827407658100128
501 0.06773446500301361
1001 0.06353870779275894
1501 0.062131118029356
2001 0.06197088956832886
2501 0.06126018986105919
2999 0.0594196543097496
Saving
1 0.3826190233230591
501 0.06429196149110794
1001 0.06024046242237091
1501 0.05867232754826546
2001 0.057827189564704895
2501 0.057516541332006454
2999 0.057248879224061966
Saving
Finished 496 1055071_20227_2
1 0.3564467430114746
501 0.06019939109683037
1001 0.05661826580762863
1501 0.055696066468954086
2001 0.054767780005931854
2501 0.054238948971033096
2999 0.05417437106370926
Saving
1 0.3515469431877136
501 0.05862663313746452
1001 0.05630863457918167
1501 0.055246781557798386
2001 0.05406755954027176
2501 0.05287710577249527
2999 0.052651286125183105
Saving
Finished 497 1055284_20227_2
1 0.3857593536376953
501 0.0631086379289627

501 0.04232063144445419
1001 0.03946702927350998
1501 0.03912542387843132
2001 0.039116475731134415
2501 0.03911616653203964
2999 0.03911615163087845
Saving
1 0.3293355703353882
501 0.04969222471117973
1001 0.043442487716674805
1501 0.042774248868227005
2001 0.04159478843212128
2501 0.040531136095523834
2999 0.0399181991815567
Finished 517 1058030_20227_3
1 0.37373167276382446
501 0.054541073739528656
1001 0.048758797347545624
1501 0.048036858439445496
2001 0.04797773063182831
2501 0.047974321991205215
2999 0.04797404631972313
Saving
1 0.3704933822154999
501 0.05819086730480194
1001 0.05143393948674202
1501 0.04809851571917534
2001 0.04683408513665199
2501 0.04654623568058014
2999 0.04526059702038765
Saving
Finished 518 1058172_20227_2
1 0.38608264923095703
501 0.06689916551113129
1001 0.06383335590362549
1501 0.06195971369743347
2001 0.06095089018344879
2501 0.05925707891583443
2999 0.05806342884898186
Saving
1 0.3892900049686432
501 0.06304803490638733
1001 0.06056191399693489
1501 0

501 0.07204411923885345
1001 0.06778693199157715
1501 0.06597895175218582
2001 0.06566175073385239
2501 0.06507714092731476
2999 0.0625777319073677
Finished 538 1060198_20227_2
1 0.3773733973503113
501 0.07161912322044373
1001 0.06707152724266052
1501 0.0662047490477562
2001 0.06597884744405746
2501 0.06596606969833374
2999 0.06596606224775314
Saving
1 0.3790813982486725
501 0.0703761950135231
1001 0.06693548709154129
1501 0.06636492162942886
2001 0.06603333353996277
2501 0.06430264562368393
2999 0.06077566742897034
Saving
Finished 539 1060248_20227_2
1 0.35007280111312866
501 0.04868067428469658
1001 0.046603813767433167
1501 0.044543709605932236
2001 0.04360482096672058
2501 0.04320342838764191
2999 0.04214023798704147
Saving
1 0.34906643629074097
501 0.05227343738079071
1001 0.04809467867016792
1501 0.046909574419260025
2001 0.045487385243177414
2501 0.04493090882897377
2999 0.04489775747060776
Finished 540 1060304_20227_2
1 0.3810710310935974
501 0.07171305269002914
1001 0.06056623

501 0.0651521384716034
1001 0.06101083382964134
1501 0.060701750218868256
2001 0.06011853739619255
2501 0.05921014025807381
2999 0.058922432363033295
Saving
1 0.3951362371444702
501 0.06371866911649704
1001 0.06125966086983681
1501 0.058692313730716705
2001 0.05777446925640106
2501 0.05687989294528961
2999 0.0564858540892601
Saving
Finished 560 1061856_20227_3
1 0.36373406648635864
501 0.06833392381668091
1001 0.06554320454597473
1501 0.06484729796648026
2001 0.06445829570293427
2501 0.06210245564579964
2999 0.06171141564846039
Saving
1 0.363202303647995
501 0.0674140453338623
1001 0.06400769203901291
1501 0.06348095089197159
2001 0.062033992260694504
2501 0.06046801432967186
2999 0.05882531777024269
Saving
Finished 561 1062047_20227_2
1 0.35866764187812805
501 0.07012159377336502
1001 0.06655954569578171
1501 0.0632912889122963
2001 0.06276091188192368
2501 0.062433283776044846
2999 0.061781588941812515
Saving
1 0.3625176250934601
501 0.07715734839439392
1001 0.06922153383493423
1501 

501 0.06270075589418411
1001 0.06040450185537338
1501 0.05772073194384575
2001 0.056948915123939514
2501 0.056868281215429306
2999 0.052748966962099075
Finished 581 1064077_20227_2
1 0.3503848612308502
501 0.060142919421195984
1001 0.055009447038173676
1501 0.05381230264902115
2001 0.05282756686210632
2501 0.052541401237249374
2999 0.052462249994277954
Saving
1 0.3479786813259125
501 0.05945896729826927
1001 0.0574808306992054
1501 0.05645502731204033
2001 0.05601666495203972
2501 0.055645428597927094
2999 0.0546654611825943
Finished 582 1064086_20227_3
1 0.3291967809200287
501 0.06374654918909073
1001 0.05981561541557312
1501 0.05610514432191849
2001 0.05449646711349487
2501 0.054286275058984756
2999 0.054271794855594635
Saving
1 0.33866867423057556
501 0.06314632296562195
1001 0.05772267282009125
1501 0.05567074194550514
2001 0.05516841635107994
2501 0.05392115190625191
2999 0.05369725450873375
Saving
Finished 583 1064111_20227_2
1 0.3995785415172577
501 0.07998830080032349
1001 0.07

In [40]:
import pickle

pickle.dump(bases, open('/home/anton/Documents/Tulane/Research/UKBB/Cypress/ab/AngleBasisLong5_0-600.pkl', 'wb'))

print('Complete')

Complete


In [4]:
import re
import numpy as np
import pickle
from math import isnan

bases = pickle.load(open('/home/anton/Documents/Tulane/Research/UKB/AngleBasis1_0-600.pkl', 'rb'))

pat = re.compile('(\d+)_(\d+)_(\d)')
thetas = []
jitter = []
age = []
sex = []
fc = []

a,b = np.triu_indices(264,1)

for sub_task_ses,basis in bases.items():
    mat = pat.search(sub_task_ses)
    sub, task, ses = mat.group(1), mat.group(2), mat.group(3)
    ag = float(df[df['f.eid'] == int(sub)]['f.21003.0.0'])
    s = float(df[df['f.eid'] == int(sub)]['f.22001.0.0'])
    if isnan(ag) or isnan(s):
        print(sub,ses)
        continue
    thetas.append(basis['thetas'])
    jitter.append(basis['jitter'])
    age.append(ag)
    sex.append(int(s))
    for items in ts[sub]:
        if items[0] == ses and items[1] == task:
            p = np.corrcoef(items[2])
            fc.append(p[a,b])
        
thetas = np.stack(thetas)
jitter = np.stack(jitter)
fc = np.stack(fc)
age = np.array(age)
sex = np.array(sex)
        
print([item.shape for item in [thetas, jitter, fc, age, sex]])
    
def tops(thetas, jitter):
    t0 = np.expand_dims(thetas, 2)
    t1 = np.expand_dims(thetas, 3)
    j0 = np.expand_dims(jitter, 2)
    j1 = np.expand_dims(jitter, 3)
    ps = np.cos(t0-t1)*(j0*j1)
    a,b = np.triu_indices(264, 1)
    ps = ps[:,:,a,b]
    return ps
    
ps = tops(thetas, jitter)
aps = np.mean(ps, axis=1)
res = fc - aps

print(ps.shape)
print(aps.shape)
print(res.shape)

1006604 2
1009819 2
1011256 2
1013566 2
1015850 2
1028465 2
1037381 2
1041164 2
1053570 2
1054850 2
1059329 2
1059905 2
1059905 3
1060674 2
[(586, 1, 264), (586, 1, 264), (586, 34716), (586,), (586,)]
(586, 1, 34716)
(586, 34716)
(586, 34716)


In [5]:
lsdir = '/home/anton/Documents/Tulane/Research/LatentSimilarity/'

import sys
if lsdir not in sys.path:
    sys.path.append(lsdir)
from latsim import LatSimReg, LatSimClf

print('Done')

Done


In [35]:
# Predict age

from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split

def rmse(yhat, y):
    if isinstance(yhat, np.ndarray) or isinstance(yhat, int):
        f = np.mean
    else:
        f = torch.mean
    return f((y-yhat)**2)**0.5

accs = []
nulls = []

for _ in range(50):
    xtr, xt, ytr, yt = train_test_split(fc, age, train_size=0.8)

    mu = np.mean(ytr)
    ytr -= mu
    yt -= mu

    reg = LatSimReg().fit(xtr, ytr) #Ridge(alpha=1).fit(xtr, ytr)
    yhat = reg.predict(xt)
    print(rmse(yhat, yt))
    print(rmse(0, yt))
    accs.append(rmse(yhat, yt))
    nulls.append(rmse(0, yt))
    
print('---')
print(np.mean(accs),np.std(accs))
print(np.mean(nulls),np.std(nulls))

7.439146237061834
7.368753833440986
7.289623041588073
6.958675244399956
8.359501367538948
7.75897374359
7.059785443257847
7.433651428047928
8.228397647308396
8.515658887073375
7.306995847750469
7.542947283324222
8.097003719170045
7.9403978083828965
8.29542052672633
8.293063167142456
8.033455471209736
7.829443527506487
7.677120762959252
7.4139916864852715
7.083634011867173
7.465845094509169
7.945594904816221
7.921312396481066
7.513845571777647
8.160497473087316
8.110122010795951
8.385358402429205
8.076820669947093
6.972916911016069
8.143452875026611
7.551043931987809
7.767837068955897
7.125672362070138
8.03707155069591
7.780991088161558
8.663842100811056
7.840909090909091
7.943863464423164
6.942742509536057
8.26268584318644
8.222530024268686
7.838650822778188
7.984535673247091
8.733542708815614
7.792199161250632
7.409182558133966
7.720365897208295
6.840621250212442
7.943531335787921
8.054265780723872
7.769309455896811
8.20242419461417
7.830355216075282
7.970792834875107
7.92156930983870

In [28]:
# Predict sex

from latsim import LatSimClf
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

accs = []
nulls = []

for _ in range(50):
    xtr, xt, ytr, yt = train_test_split(aps, sex, stratify=sex, train_size=0.8)

    reg = LatSimClf(d=2).fit(xtr, ytr) #LogisticRegression(C=1, max_iter=1000).fit(xtr, ytr)
    yhat = reg.predict(xt)
    print(np.mean(yhat == yt))
    print(np.mean(yt))
    accs.append(np.mean(yhat == yt))
    nulls.append(np.mean(yt))
    
print('---')
print(np.mean(accs),np.std(accs))
print(np.mean(nulls),np.std(nulls))

0.6101694915254238
0.4661016949152542
0.6694915254237288
0.4661016949152542
0.7203389830508474
0.4661016949152542
0.7627118644067796
0.4661016949152542
0.6101694915254238
0.4661016949152542
0.6949152542372882
0.4661016949152542
0.635593220338983
0.4661016949152542
0.6101694915254238
0.4661016949152542
0.652542372881356
0.4661016949152542
0.6186440677966102
0.4661016949152542
0.6949152542372882
0.4661016949152542
0.6101694915254238
0.4661016949152542


KeyboardInterrupt: 